In [1]:
from src.forecast.cnn import cnnpred_2d, datagen, f1macro, testgen, scale_inputs, precision_m
from src.utils.utils import load, add_variables

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

## Testowanie CNN 2D

In [4]:
CURR = "EURUSD"
INTERVAL = "4h"

In [5]:
# load data
eurusd = load(currency=CURR, interval=INTERVAL)
eurusd = add_variables(ohlc=eurusd)

# clean data
eurusd.drop(columns=['target_value', 'target_direction'], inplace=True)


# scale inputs
eurusd = scale_inputs(eurusd, targetcol=["impulse"]) # use different scaler to preserver more info between small values

### Training model

In [6]:
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.h5"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max", verbose=0,
                    save_best_only=True, save_weights_only=False, save_freq="epoch")
]

In [54]:
seq_len = 10
batch_size = 64
n_epochs   = 4
n_features = 49

class_weight={
        0: 1,
        1: 1
}
 
model = cnnpred_2d(seq_len=seq_len, n_features=n_features)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), 
              loss=tf.losses.BinaryCrossentropy(),
              metrics=["acc", f1macro, precision_m])

model.fit(datagen(df=eurusd,
                  seq_len=seq_len,
                  batch_size=batch_size,
                  targetcol=["impulse"],
                  kind="train"),
          validation_data=datagen(df=eurusd,
                                  seq_len=seq_len,
                                  batch_size=batch_size,
                                  targetcol=["impulse"],
                                  kind="valid"),
          epochs=n_epochs, 
          steps_per_epoch=200, 
          validation_steps=10,
          class_weight=class_weight,
          verbose=1,
          callbacks=callbacks)

Epoch 1/4


2023-03-23 00:06:27.150434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - ETA: 0s - loss: 0.2077 - acc: 0.9508 - f1macro: 0.4874 - precision_m: 2.5000e-04

2023-03-23 00:06:35.137038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 9s 40ms/step - loss: 0.2077 - acc: 0.9508 - f1macro: 0.4874 - precision_m: 2.5000e-04 - val_loss: 0.2204 - val_acc: 0.9438 - val_f1macro: 0.4854 - val_precision_m: 0.0000e+00
Epoch 2/4
200/200 [==============================] - 8s 39ms/step - loss: 0.2020 - acc: 0.9493 - f1macro: 0.4869 - precision_m: 0.0000e+00 - val_loss: 0.2171 - val_acc: 0.9438 - val_f1macro: 0.4855 - val_precision_m: 0.0000e+00
Epoch 3/4
200/200 [==============================] - 8s 39ms/step - loss: 0.2017 - acc: 0.9491 - f1macro: 0.4868 - precision_m: 0.0000e+00 - val_loss: 0.2449 - val_acc: 0.9344 - val_f1macro: 0.4830 - val_precision_m: 0.0000e+00
Epoch 4/4
200/200 [==============================] - 8s 38ms/step - loss: 0.2001 - acc: 0.9495 - f1macro: 0.4870 - precision_m: 0.0000e+00 - val_loss: 0.1987 - val_acc: 0.9500 - val_f1macro: 0.4871 - val_precision_m: 0.0000e+00


In [55]:
# Prepare test data
test_data, test_target = testgen(eurusd, seq_len, ["impulse"])

# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.1).astype(int)
print(f"accuracy: {accuracy_score(test_pred, test_target)}")
print(f"MAE: {mean_absolute_error(test_pred, test_target)}")
print(f"F1: {f1_score(test_pred, test_target)}")

 41/330 [==>...........................] - ETA: 0s

2023-03-23 00:07:02.350865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


330/330 [==============================] - 1s 2ms/step
accuracy: 0.9497915877226222
MAE: 0.050208412277377795
F1: 0.0


In [56]:
test_out

array([[0.0541468],
       [0.0541468],
       [0.0541468],
       ...,
       [0.0541468],
       [0.0541468],
       [0.0541468]], dtype=float32)

In [57]:
eurusd['impulse'].value_counts()/eurusd.shape[0]

0    0.949141
1    0.050859
Name: impulse, dtype: float64

In [45]:
pd.DataFrame(test_pred).value_counts()

1    10556
dtype: int64